# Segmenting and Clustering Neighborhoods in Toronto - 2nd Part

This is the notebook that contains the code for the __second part__ of the __week 3 assignment__ of the course __Applied Data Science Capstone__. 

In order to parse the HTML page, I decided to use *BeautifulSoup*.

In [1]:
# Install beautifulsoup4 if necessary
# !conda install -c anaconda beautifulsoup4 --yes 

# import the required libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
# The wikipedia URL which has the information 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
# Read the URL page at Wikipedia
data = BeautifulSoup(page.text, "html.parser")

# Get the HTML table inside the HTML data read
table = data.find('table')

# Get all the rows of the HTML table
table_rows = table.find_all('tr')

# Get the header TH
header = table_rows[0].find_all('th')

# Get the column values (TD) of the header TH, which will be the column labels of the dataframe
labels = [table_rows[0].text for table_rows[0] in header]
# Clean the '\n' special character
labels = np.char.replace(labels, '\n', '')

# Create the dataframe
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df = pd.DataFrame(columns = labels)

# Get all the other columns (TD) from the rows (TR)
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    # Ignore the first row (header) and the rows which have "Not assigned" boroughs
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
    if (row != [] and row[1] != "Not assigned\n"):
        row = pd.Series(row,labels)
        df = df.append([row],ignore_index=True)     

# Clean the data removing '\n'        
df.replace('\n','', regex=True, inplace=True)   

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
arr = np.where(pd.isnull(df['Neighborhood']))
print(arr)
# there are no 'not assigned' neighborhoods in the dataframe

df.head()

(array([], dtype=int64),)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
# Read the geospatial data csv
geospatial = pd.read_csv("http://cocl.us/Geospatial_data") 
geospatial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [4]:
# Change the dataframes indexes, in order to make it possible to merge them later
df.set_index('Postal Code', inplace=True)
geospatial.set_index('Postal Code', inplace=True)

In [5]:
# Merge both dataframes
mergedDf = df.merge(geospatial, left_index=True, right_index=True)
mergedDf

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...
M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558


In [6]:
# Transform Postal Code into a column again
mergedDf['CEP'] = mergedDf.index
mergedDf.reset_index(inplace=True)
del mergedDf['Postal Code']
mergedDf.rename(columns={'CEP':'Postal Code'}, inplace=True)

mergedDf.head()

,Borough,Neighborhood,Latitude,Longitude,Postal Code
0,North York,Parkwoods,43.753259,-79.329656,M3A
1,North York,Victoria Village,43.725882,-79.315572,M4A
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,M5A
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,M6A
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,M7A


In [7]:
# Define Postal Code as the first column
cols = mergedDf.columns.tolist()
cols = cols[-1:] + cols[:-1]
mergedDf = mergedDf[cols]
mergedDf.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Below is the final result: a dataframe with 5 columns and 103 rows

In [8]:
mergedDf

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
